# Getting data ready

In [125]:
#essentials
import os
import numpy as np
import pandas as pd
import dataframe_image as dfi
from tqdm import tqdm
import pickle
import warnings
warnings.filterwarnings('ignore')

#visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#tools/metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split
import shap

#modeling
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

#pandas show all columns
pd.set_option('display.max_columns', None)

#%load_ext autoreload
#%autoreload 2
#%run ./__init__

In [126]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from fitter import Fitter, get_common_distributions, get_distributions, pd.options.display.max_rows

In [127]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error


In [128]:
kp =  pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\KenPom Team Metrics.csv")
kp_conf_rat =  pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\KP_Conf_Ratings.csv")


In [129]:
kp.head(2)

,Season,TeamName,TeamName 2,Tempo,RankTempo,AdjTempo,RankAdjTempo,OE,RankOE,AdjOE,RankAdjOE,DE,RankDE,AdjDE,RankAdjDE,AdjEM,RankAdjEM,seed
0,2019,ABILENE CHRISTIAN,Abilene Christian,66.5681,293,66.0042,254,107.1020,87,103.625,186,96.7466,43,101.539,119,2.08626,145,15.0
1,2019,AIR FORCE,Air Force,67.2794,269,66.4070,234,98.7687,274,100.431,250,105.6820,242,107.275,238,-6.84432,241,NaN


In [130]:
kp_conf_rat.head(2)

,Year_Conf,Conf_Abb,Conf_Rk,Conf_Full,Conf_Rating,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,2023,Big 12,1,Big 12 Conference,17.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023,Big Ten,2,Big Ten Conference,14.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
#data_path = os.path.dirname(os.getcwd()) + '/data' + '/master_table.csv'
#master_table = pd.read_csv(data_path)
#master_table = pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\final_dataset.csv")

In [132]:
#master_table.head()

In [133]:
#list(master_table.columns)

In [134]:
d = pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\final_data_by_player_type.csv")
d = pd.merge(d, kp, left_on=['Team', 'year'], right_on=['TeamName', 'Season'], how='left')
d = pd.merge(d, kp, left_on=['Team_n1', 'year'], right_on=['TeamName', 'Season'], how='left')
df = pd.merge(d, kp, left_on=['Team_n1', 'year_n1'], right_on=['TeamName', 'Season'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year_n1'], right_on=['Conf_Abb', 'Year_Conf'], how='left')

df = df.replace(to_replace="NO IDEA",value=0)
df = df.replace(to_replace="---",value=0)
df = df.fillna(0)
pd.options.display.max_columns

df['offense_ORtg'] = df['X.Min_n1']*df['ORtg_n1']
df['offense'] = df['X.Min_n1']*df['bpm_n1']  #box plus minus
# (stl_per_poss+blk_per_poss+Dreb%)*%Min
df['defence_standard'] = ((df['stl_per_poss_n1'] + df['blk_per_poss_n1'] + df['drb_pct_n1']) * df['X.Min_n1'])+1
df['defence_dws'] = ((df['stl_per_poss_n1'] + df['blk_per_poss_n1'] + df['drb_pct_n1']) * (df['dws_n1'] +1 )+1)
df['defence_standard_dws'] = ( df['defence_standard']  * (df['dws_n1'] +1 ))
df['defence_def_rtg'] = ((df['stl_per_poss_n1'] + df['blk_per_poss_n1'] + df['drb_pct_n1']) * (df['def_rtg_n1'] +1 )+1)
df['defence_standard_def_rtg'] = ( df['defence_standard']  * df['def_rtg_n1'])+1
df['defence_all'] = ((df['stl_per_poss_n1'] + df['blk_per_poss_n1'] + df['drb_pct_n1']) * df['X.Min_n1'] * df['def_rtg_n1'] * (1+ df['dws_n1']))+100

In [135]:
df[['X.Min_n1','mp_per_g','games', 'stl_per_poss_n1','def_rtg_n1', 'dws_n1','defence_def_rtg','offense', 'offense_ORtg']].describe()

,X.Min_n1,mp_per_g,games,stl_per_poss_n1,def_rtg_n1,dws_n1,defence_def_rtg,offense,offense_ORtg
count,11042.000000,11042.000000,11042.000000,11042.000000,11042.000000,11042.000000,11042.000000,11042.000000,11042.000000
mean,48.420766,19.833680,25.684749,1.804238,102.183862,0.830882,1686.401753,25.341693,4985.366356
std,25.011647,8.984115,9.599929,0.899598,7.583201,0.723969,538.503392,216.611179,2756.265642
min,2.000000,1.600000,1.000000,0.000000,0.000000,-0.800000,1.000000,-855.540000,0.000000
25%,27.825000,12.100000,20.000000,1.200000,98.400000,0.300000,1294.960000,-103.622500,2643.690000
50%,51.500000,20.300000,28.000000,1.700000,102.400000,0.700000,1646.080000,-13.095000,5212.430000
75%,69.700000,27.400000,31.000000,2.300000,106.400000,1.200000,2038.070000,122.182500,7268.787500
max,96.500000,38.700000,144.000000,10.500000,132.700000,8.600000,4086.250000,1259.200000,11709.600000


In [136]:
df = df.drop(df[(df['games']<=13) & (df['games_started']/df['games']>0.1)].index)
df = df.drop(df[(df['games_n1']<=13) & (df['games_started_n1']/df['games_n1']>0.1)].index)


df = df.drop(df.loc[((df['games']>39) & (df['year']==2023)) | ((df['games_n1']>39) & (df['year_n1']==2023)) | 
   ((df['games']>40) & (df['year']==2022)) | ((df['games_n1']>40) & (df['year_n1']==2022)) | 
   ((df['games']>33) & (df['year']==2021)) | ((df['games_n1']>33) & (df['year_n1']==2021)) | 
   ((df['games']>35) & (df['year']==2020)) | ((df['games_n1']>35) & (df['year_n1']==2020)) | 
   ((df['games']>40) & (df['year']==2019)) | ((df['games_n1']>40) & (df['year_n1']==2019)) |
   (df['year']==0)]['Player.Name'].index)
idx = df.groupby(['Team', 'year'])['games'].max()
a = pd.DataFrame(idx)
ab = a.reset_index()
ab.columns = ['Team_max', 'year_max', 'games_max']
df = pd.merge(df, ab, left_on=['Team', 'year'], right_on=['Team_max', 'year_max'], how='left')
idx2 = df.groupby(['Team_n1', 'year_n1'])['games_n1'].max()
a2 = pd.DataFrame(idx2)
ab2 = a2.reset_index()
ab2.columns = ['Team_max', 'year_max', 'games_max']
df = pd.merge(df, ab2, left_on=['Team_n1', 'year_n1'], right_on=['Team_max', 'year_max'], how='left')

In [137]:
df = df.drop(df.loc[(df['games_max_x']<15)].index)
df = df.drop(df.loc[(df['games_max_y']<15)].index)

#df = df.drop(df.loc[(df['year']==2020) | (df['year_n1']==2020)].index)
#df = df.drop(df[(df['X.Min_n1']<=10)].index)
#df = df.drop(df[(df['X.Min']<=10)].index)

In [138]:
# df = df.drop(df[(df['games']<=0.05*df['games_max_x'])].index)
# df = df.drop(df[(df['games_n1']<=0.05*df['games_max_y'])].index)

In [139]:
df = df.drop(df[(df['X.Min_n1']<=6)].index)
df = df.drop(df[(df['X.Min']<=6)].index)
df['Year_dif'] = df['year_n1']-df['year']
df['Team_AdjEM_dif'] = df['AdjEM_y']-df['AdjEM_x']
df['KP_Rk_Dif'] = df['RankAdjEM_y']-df['RankAdjEM_x']
df['KP_Conf_Dif'] = df['Conf_Rating_y']-df['Conf_Rating_x']
df['Calc A'] = df['X.Min']*df['ws']
df['Calc A2'] = df['X.Min']*df['bpm']
df['Calc A3'] = df['X.Min']*df['ORtg']
df['Calc A2_n1'] = df['X.Min_n1']*df['bpm_n1']
df['Calc A_n1'] =df['X.Min_n1']*df['ws_n1']
df['Calc A3_n1'] = df['X.Min_n1']*df['ORtg_n1']
df['Calc B'] = (df['stl_per_poss']+df['blk_per_poss']+df['DR.'])*df['X.Min']
df['Calc B_n1'] = (df['stl_per_poss_n1']+df['blk_per_poss_n1']+df['DR._n1'])*df['X.Min_n1']
df['Calc C']=(df['off_rtg']-df['def_rtg'])*df['X.Min']
df['Calc C_n1']=(df['off_rtg_n1']-df['def_rtg_n1'])*df['X.Min_n1']
df['Calc Overall A'] = df['Calc A'] + df['Calc B']
df['Calc Overall A2'] = df['Calc A2'] + df['Calc B']
df['Calc Overall A3'] = df['Calc A3'] + df['Calc B']
df['Calc Overall A_n1'] = df['Calc A_n1'] + df['Calc B_n1']
df['Calc Overall A2_n1'] = df['Calc A2_n1'] + df['Calc B_n1']
df['Calc Overall A3_n1'] = df['Calc A3_n1'] + df['Calc B_n1']

In [140]:
df[['X.Min_n1','mp_per_g','offense', 'bpm_n1', 'dws_n1',
    'defence_standard','defence_dws','defence_def_rtg','defence_all']].describe()

,X.Min_n1,mp_per_g,offense,bpm_n1,dws_n1,defence_standard,defence_dws,defence_def_rtg,defence_all
count,7747.000000,7747.000000,7747.000000,7747.000000,7747.000000,7747.000000,7747.000000,7747.000000,7.747000e+03
mean,53.556344,20.915787,40.479528,0.029315,0.860372,888.076017,32.761253,1686.551812,1.829134e+05
std,22.218201,8.455224,230.559896,4.189242,0.563192,481.672397,17.024719,528.339573,1.342275e+05
min,6.100000,2.500000,-765.850000,-29.800000,-0.800000,1.000000,1.000000,1.000000,1.000000e+02
25%,36.500000,13.700000,-108.215000,-2.800000,0.400000,522.580000,20.640000,1301.500000,7.969390e+04
50%,56.400000,21.400000,0.000000,0.000000,0.800000,845.220000,29.350000,1649.500000,1.548366e+05
75%,72.000000,28.000000,164.860000,2.900000,1.200000,1188.120000,40.580000,2035.190000,2.572467e+05
max,96.500000,38.500000,1208.900000,15.700000,5.500000,3326.440000,148.420000,4086.250000,1.074217e+06


In [141]:
#len(df[df['bpm_n1']<-10].index)   # 46
df = df.drop(df[(df['bpm_n1']<-10)].index)

In [142]:
#df = df[(df['X.Min_n1']>6) & (df['games']<40)  & (df['games']>7)]
#filtered_df
df[['X.Min_n1','mp_per_g','games', 'defence_standard' ,'offense', 'bpm_n1']].describe()

,X.Min_n1,mp_per_g,games,defence_standard,offense,bpm_n1
count,7681.000000,7681.000000,7681.000000,7681.000000,7681.000000,7681.000000
mean,53.863625,20.984247,26.958990,893.527610,42.656832,0.132626
std,22.039255,8.435249,6.033393,479.891972,229.956863,4.048110
min,6.100000,2.500000,2.000000,1.000000,-765.850000,-10.000000
25%,37.000000,13.800000,23.000000,530.740000,-106.560000,-2.700000
50%,56.700000,21.500000,29.000000,849.640000,0.000000,0.000000
75%,72.100000,28.100000,32.000000,1192.190000,166.760000,2.900000
max,96.500000,38.500000,40.000000,3326.440000,1208.900000,15.700000


In [143]:
df.to_csv("D:\\Github\\Capstone-project\\Data folder\\training_data_to_use.csv", index=False)

In [144]:
df1 =  pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\training_data_to_use.csv")

In [145]:
df[['X.Min_n1','mp_per_g','games', 'defence_standard' ,'offense', 'bpm_n1', 'dws_n1', 'defence_dws' ,'defence_all']].describe()

,X.Min_n1,mp_per_g,games,defence_standard,offense,bpm_n1,dws_n1,defence_dws,defence_all
count,7681.000000,7681.000000,7681.000000,7681.000000,7681.000000,7681.000000,7681.000000,7681.000000,7.681000e+03
mean,53.863625,20.984247,26.958990,893.527610,42.656832,0.132626,0.866619,32.890956,1.842197e+05
std,22.039255,8.435249,6.033393,479.891972,229.956863,4.048110,0.561411,17.028222,1.340441e+05
min,6.100000,2.500000,2.000000,1.000000,-765.850000,-10.000000,-0.800000,1.000000,1.000000e+02
25%,37.000000,13.800000,23.000000,530.740000,-106.560000,-2.700000,0.400000,20.800000,8.153551e+04
50%,56.700000,21.500000,29.000000,849.640000,0.000000,0.000000,0.800000,29.500000,1.565997e+05
75%,72.100000,28.100000,32.000000,1192.190000,166.760000,2.900000,1.200000,40.690000,2.582629e+05
max,96.500000,38.500000,40.000000,3326.440000,1208.900000,15.700000,5.500000,148.420000,1.074217e+06
